<a href="https://colab.research.google.com/github/diegomrodrigues/my_gpt/blob/main/GPT-2%20Fine-Tunned%20for%20Sentence%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""Como aproveitar ao máximo sua assinatura do Colab

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/github/diegomrodrigues/my_gpt/blob/main/GT2%20Starter%20Implementation.ipynb
"""

!pip install tiktoken datasets --quiet

import torch
import math
from torch import nn

def gelu(x):
    return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-12):
        super(LayerNorm, self).__init__()

        # Parâmetros aprendíveis para escala (gamma) e deslocamento (beta)
        # Inicializados com 1s e 0s respectivamente
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))

        # Epsilon para estabilidade numérica
        self.epsilon = eps

    def forward(self, x):  # x: [batch_size, seq_length, hidden_size]
        # Calcula a média ao longo da última dimensão (feature dimension)
        # keepdim=True mantém a dimensionalidade para broadcast correto
        mu = x.mean(-1, keepdim=True)  # [batch_size, seq_length, 1]

        # Calcula a variância
        # Usa a fórmula E[(X - μ)^2] para variância
        sigma = (x - mu).pow(2).mean(-1, keepdim=True)  # [batch_size, seq_length, 1]

        # Normalização: (x - μ) / sqrt(σ^2 + ε)
        # Epsilon (ε) evita divisão por zero
        x = (x - mu) / torch.sqrt(sigma + self.epsilon)  # [batch_size, seq_length, hidden_size]

        # Aplica transformação afim com parâmetros aprendíveis
        # y = γ * x + β, onde γ = self.weight e β = self.bias
        return self.weight * x + self.bias  # [batch_size, seq_length, hidden_size]

class Conv1D(nn.Module):
    def __init__(self, nf, nx):
        # nf: número de filtros (saída)
        # nx: tamanho da entrada
        super(Conv1D, self).__init__()

        self.nf = nf

        # Inicializa os pesos com uma distribuição normal
        # [nx, nf]
        w = torch.empty(nx, nf)
        nn.init.normal_(w, std=0.02)

        # Cria parâmetros treináveis para pesos e vieses
        self.weight = nn.Parameter(w)  # [nx, nf]
        self.bias = nn.Parameter(torch.zeros(nf))  # [nf]

    def forward(self, x): # x: [batch_size, input_len, nx]

        # Prepara o shape de saída
        size_out = x.size()[:-1] + (self.nf,) # [batch_size, input_len, nf]

        # Reshape x para 2D
        x_2d = x.view(-1, x.size(-1)) # [batch_size * input_len, nx]

        # Aplica a transformação linear
        # torch.addmm realiza: out = beta * self.bias + alpha * (x_2d @ self.weight)
        x_transformed = torch.addmm(self.bias, x_2d, self.weight) # [batch_size * input_len, nf]

        # Reshape de volta para 3D
        x_output = x_transformed.view(*size_out) # [batch_size, input_len, nf]

        return x_output

class SingleHeadAttention(nn.Module):
    def __init__(self, nx, n_ctx, scale=False):
        super(SingleHeadAttention, self).__init__()

        # nx: dimensão do modelo (tamanho dos embeddings)
        self.nx = nx
        # n_ctx: comprimento máximo do contexto (número máximo de tokens na sequência)
        self.n_ctx = n_ctx

        # Performar scaled dot product?
        self.scale = scale

        # Criamos uma máscara de atenção triangular inferior (causal)
        # Isso garante que cada token só preste atenção aos tokens anteriores
        # Shape: [1, 1, n_ctx, n_ctx]
        self.register_buffer("bias", torch.tril(torch.ones(n_ctx, n_ctx)).view(1, 1, n_ctx, n_ctx))

        # Camada linear para projetar a entrada em Query, Key, Value
        # Multiplica por 3 porque criamos Q, K, V de uma vez
        self.c_attn = Conv1D(nx * 3, nx)
        # Camada linear para projetar a saída da atenção de volta ao espaço do modelo
        self.c_proj = Conv1D(nx, nx)

    def _attention(self, q, k, v):
        # Calculamos os scores de atenção: Query * Key^T
        # Isso mede quanto cada token (Query) deve prestar atenção a cada outro token (Key)
        w = torch.matmul(q, k)  # [batch_size, 1, n_ctx, n_ctx]

        # Aplicamos escala opcional para estabilizar o treinamento
        # Dividimos pelo sqrt da dimensão para evitar que os gradientes fiquem muito grandes
        if self.scale:
            w = w / math.sqrt(v.size(-1))

        # Preparamos a máscara causal
        # Isso garante que não olhamos para tokens futuros
        nd, ns = w.size(-2), w.size(-1)
        mask = self.bias[:,:,ns-nd:ns,:ns]

        # Aplicamos a máscara causal
        # Colocamos -infinito onde a máscara é 0, efetivamente zerando esses scores no softmax
        w = w * mask - 1e10 * (1 - mask)

        # Aplicamos softmax para obter os pesos de atenção
        # Isso normaliza os scores para que somem 1 para cada query
        w = nn.Softmax(dim=-1)(w)

        # Calculamos a saída da atenção: pesos de atenção * Values
        # Isso agrega as informações dos tokens relevantes
        output = torch.matmul(w, v)  # [batch_size, 1, n_ctx, nx]

        return output

    def forward(self, x, layer_past=None):
        # x: entrada [batch_size, n_ctx, nx]

        # Projetamos a entrada para Query, Key, Value de uma vez
        qkv = self.c_attn(x)  # [batch_size, n_ctx, nx*3]

        # Separamos Q, K, V
        query, key, value = qkv.split(self.nx, dim=2)  # cada um: [batch_size, n_ctx, nx]

        # Reshape para adicionar dimensão de cabeça (neste caso, apenas 1)
        # Isso prepara os tensores para a operação de atenção
        query = query.unsqueeze(1)  # [batch_size, 1, n_ctx, nx]
        key = key.unsqueeze(1).transpose(-1, -2)  # [batch_size, 1, nx, n_ctx]
        value = value.unsqueeze(1)  # [batch_size, 1, n_ctx, nx]

        # Lidamos com o cache do estado passado, se fornecido
        # Isso é útil para geração incremental de texto
        if layer_past:
            past_key, past_value = layer_past
            key = torch.cat((past_key, key), dim=-1)
            value = torch.cat((past_value, value), dim=-2)

        # Armazenamos o estado atual para uso futuro
        present = torch.stack((key.transpose(-1, -2), value))

        # Calculamos a atenção
        attn_output = self._attention(query, key, value)  # [batch_size, 1, n_ctx, nx]

        # Removemos a dimensão da cabeça (que era 1)
        attn_output = attn_output.squeeze(1)  # [batch_size, n_ctx, nx]
        # Aplicamos a projeção final para voltar ao espaço do modelo
        attn_output = self.c_proj(attn_output)  # [batch_size, n_ctx, nx]

        return attn_output, present

class MultiHeadAttention(nn.Module):
    def __init__(self, nx, n_ctx, n_head, scale=False):
        super(MultiHeadAttention, self).__init__()

        # Cria uma máscara de atenção triangular inferior (causal)
        # Isso garante que cada token só preste atenção aos tokens anteriores
        # Shape: [1, 1, n_ctx, n_ctx]
        self.register_buffer("bias", torch.tril(torch.ones(n_ctx, n_ctx)).view(1, 1, n_ctx, n_ctx))

        self.n_head = n_head  # Número de cabeças de atenção
        self.split_size = nx  # Tamanho da dimensão do modelo
        self.scale = scale    # Flag para aplicar escala nos scores de atenção

        # Camada linear para projetar a entrada em Query, Key, Value para todas as cabeças
        self.c_attn = Conv1D(nx * 3, nx)
        # Camada linear para projetar a saída da atenção de volta ao espaço do modelo
        self.c_proj = Conv1D(nx, nx)

    def _attention(self, q, k, v):
        # Calcula os scores de atenção: Query * Key^T
        # Shape: [batch_size, n_head, seq_len, seq_len]
        w = torch.matmul(q, k)

        # Aplica escala opcional para estabilizar o treinamento
        if self.scale:
            w = w / math.sqrt(v.size(-1))

        # Prepara e aplica a máscara causal
        nd, ns = w.size(-2), w.size(-1)
        mask = self.bias[:, :, ns-nd:ns, :ns]
        w = w * mask - 1e10 * (1 - mask)  # Aplica -inf onde a máscara é 0

        # Aplica softmax para obter os pesos de atenção
        w = nn.Softmax(dim=-1)(w)

        # Calcula a saída da atenção: pesos de atenção * Values
        output = torch.matmul(w, v)
        return output

    def _merge_heads(self, x):
        # Reorganiza o tensor de [batch, head, seq, features] para [batch, seq, head*features]
        x = x.permute(0, 2, 1, 3).contiguous()
        new_x_shape = x.size()[:-2] + (x.size(-2) * x.size(-1),)
        return x.view(*new_x_shape)

    def _split_heads(self, x, k=False):
        # Divide o último dimensão em [n_head, features/n_head]
        new_x_shape = x.size()[:-1] + (self.n_head, x.size(-1) // self.n_head)
        x = x.view(*new_x_shape)

        # Reorganiza o tensor para [batch, head, seq, features/n_head]
        if k:
            # Para as keys, colocamos a dim seq por último para otimizar a multiplicação de matrizes
            return x.permute(0, 2, 3, 1)
        else:
            return x.permute(0, 2, 1, 3)

    def forward(self, x, layer_past=None):
        # x: entrada [batch_size, seq_len, nx]

        # Projeta a entrada para Q, K, V de uma vez
        qkv = self.c_attn(x)  # [batch_size, seq_len, nx*3]

        # Separa Q, K, V
        query, key, value = qkv.split(self.split_size, dim=2)
        # query, key, value: cada um [batch_size, seq_len, nx]

        # Divide as cabeças e reorganiza
        query = self._split_heads(query)  # [batch_size, n_head, seq_len, nx/n_head]
        key = self._split_heads(key, k=True)  # [batch_size, n_head, nx/n_head, seq_len]
        value = self._split_heads(value)  # [batch_size, n_head, seq_len, nx/n_head]

        # Lida com o cache do estado passado, se fornecido
        if layer_past:
            past_key, past_value = layer_past[0].transpose(-2, -1), layer_past[1]
            key = torch.cat((past_key, key), dim=-1)  # [batch_size, n_head, nx/n_head, seq_len_extended]
            value = torch.cat((past_value, value), dim=-2)  # [batch_size, n_head, seq_len_extended, nx/n_head]

        # Armazena o estado atual para uso futuro
        present = torch.stack((key.transpose(-2, -1), value))
        # present: [2, batch_size, n_head, seq_len, nx/n_head]

        # Calcula a atenção para todas as cabeças
        attn_output = self._attention(query, key, value)
        # [batch_size, n_head, seq_len, nx/n_head]

        # Combina as cabeças novamente
        attn_output = self._merge_heads(attn_output)  # [batch_size, seq_len, nx]

        # Projeta de volta para o espaço do modelo
        attn_output = self.c_proj(attn_output)  # [batch_size, seq_len, nx]

        return attn_output, present

class MLP(nn.Module):
    def __init__(self, n_state, n_embed):
        super(MLP, self).__init__()

        # n_state: geralmente 4 * n_embed, seguindo a arquitetura original do Transformer
        # n_embed: dimensão do modelo (embedding dimension)

        # Primeira camada linear: expande a dimensão
        self.c_fc = Conv1D(n_state, n_embed)

        # Segunda camada linear: projeta de volta para a dimensão original
        self.c_proj = Conv1D(n_embed, n_state)

        # Função de ativação GELU (Gaussian Error Linear Unit)
        self.activation = gelu

    def forward(self, x):
        # x: entrada [batch_size, seq_len, n_embed]

        # Aplica a primeira transformação linear e a função de ativação
        h = self.c_fc(x)  # [batch_size, seq_len, n_state]
        h = self.activation(h)  # [batch_size, seq_len, n_state]

        # Aplica a segunda transformação linear
        h2 = self.c_proj(h)  # [batch_size, seq_len, n_embed]

        return h2  # [batch_size, seq_len, n_embed]

class TransformerBlock(nn.Module):
    def __init__(self, n_ctx, config, scale=False):
        super(TransformerBlock, self).__init__()

        nx     = config.n_embed  # Dimensão do modelo
        n_head = config.n_head   # Número de heads

        # Primeira camada de normalização, aplicada antes da atenção
        self.ln_1 = LayerNorm(nx, eps=config.layer_norm_epsilon)

        # Camada de atenção (neste caso, atenção de cabeça única)
        if n_head <= 1:
            self.attention = SingleHeadAttention(nx, n_ctx, scale)
        else:
            self.attention = MultiHeadAttention(nx, n_ctx, n_head, scale)

        # Segunda camada de normalização, aplicada antes da MLP
        self.ln_2 = LayerNorm(nx, eps=config.layer_norm_epsilon)

        # MLP (Feed-Forward Network)
        self.mlp = MLP(4 * nx, nx)

    def forward(self, x, layer_past=None):
        # x: entrada [batch_size, seq_len, nx]

        # Primeira normalização de camada
        normalized_x = self.ln_1(x)  # [batch_size, seq_len, nx]

        # Camada de atenção
        attention_output, present = self.attention(normalized_x, layer_past=layer_past)
        # attention_output: [batch_size, seq_len, nx]
        # present: estado cacheado para geração incremental

        # Conexão residual após a atenção
        x = x + attention_output  # [batch_size, seq_len, nx]

        # Segunda normalização de camada
        normalized_x = self.ln_2(x)  # [batch_size, seq_len, nx]

        # MLP
        mlp_output = self.mlp(normalized_x)  # [batch_size, seq_len, nx]

        # Conexão residual após a MLP
        x = x + mlp_output  # [batch_size, seq_len, nx]

        return x, present

import copy
import torch
import torch.nn as nn

class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.n_layer = config.n_layer  # Número de camadas do Transformer
        self.n_embed = config.n_embed  # Dimensão do embedding
        self.n_vocab = config.n_vocab  # Tamanho do vocabulário
        self.n_pos   = config.n_pos    # Número máximo de posições

        # Embedding de tokens
        self.wte = nn.Embedding(self.n_vocab, self.n_embed)
        # Embedding de posições
        self.wpe = nn.Embedding(self.n_pos, self.n_embed)

        # Cria um bloco do Transformer
        block = TransformerBlock(config.n_ctx, config, scale=True)

        # Cria uma lista de blocos do Transformer
        self.h = nn.ModuleList([copy.deepcopy(block) for _ in range(self.n_layer)])

        # Camada final de normalização
        self.ln_f = LayerNorm(self.n_embed, eps=config.layer_norm_epsilon)

    def forward(self, input_ids, position_ids=None, token_type_ids=None, past=None):
        # input_ids: [batch_size, seq_len]

        if past is None:
            past_length = 0
            past = [None] * len(self.h)
        else:
            past_length = past[0][0].size(-2)

        if position_ids is None:
            # Gera ids de posição se não fornecidos
            position_ids = torch.arange(past_length, input_ids.size(-1) + past_length, dtype=torch.long, device=input_ids.device)
            position_ids = position_ids.unsqueeze(0).expand_as(input_ids)  # [batch_size, seq_len]

        input_shape = input_ids.size()
        input_ids = input_ids.view(-1, input_ids.size(-1))  # [batch_size * seq_len]
        position_ids = position_ids.view(-1, position_ids.size(-1))  # [batch_size * seq_len]

        # Aplica embeddings de tokens e posições
        input_embeds = self.wte(input_ids)  # [batch_size * seq_len, n_embed]
        position_embeds = self.wpe(position_ids)  # [batch_size * seq_len, n_embed]

        if token_type_ids is not None:
            token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1))
            token_type_embeds = self.wte(token_type_ids)  # [batch_size * seq_len, n_embed]
        else:
            token_type_embeds = 0

        # Soma todos os embeddings
        hidden_states = input_embeds + position_embeds + token_type_embeds  # [batch_size * seq_len, n_embed]

        presents = []
        for block, layer_past in zip(self.h, past):
            hidden_states, present = block(hidden_states, layer_past)
            # hidden_states: [batch_size * seq_len, n_embed]
            presents.append(present)

        # Aplica a normalização final
        hidden_states = self.ln_f(hidden_states)  # [batch_size * seq_len, n_embed]

        # Reshape para a forma original
        output_shape = input_shape + (hidden_states.size(-1),)
        hidden_states = hidden_states.view(*output_shape)  # [batch_size, seq_len, n_embed]

        return hidden_states, presents

import torch.nn as nn

class LinearReadoutHead(nn.Module):
    """
    Cabeça de leitura linear para o modelo GPT.
    Esta classe é responsável por projetar os estados ocultos de volta para o espaço do vocabulário.
    """

    def __init__(self, model_embedding_weights: nn.Parameter, config: 'GPTConfig'):
        super().__init__()
        # Dimensão dos embeddings
        self.n_embed = config.n_embed
        # Tamanho do vocabulário
        self.n_vocab = config.n_vocab
        self.set_embedding_weights(model_embedding_weights)

    def set_embedding_weights(self, model_embedding_weights: nn.Parameter):
        # Cria a camada linear do decodificador sem viés
        self.decoder = nn.Linear(self.n_embed, self.n_vocab, bias=False)

        # Define os pesos do decodificador como os pesos de embedding
        self.decoder.weight = model_embedding_weights

    def forward(self, hidden_state: torch.Tensor) -> torch.Tensor:
        lm_logits = self.decoder(hidden_state)
        return lm_logits

from dataclasses import dataclass, field

@dataclass
class GPTConfig:
    # Parâmetros do modelo
    n_vocab: int = 50257  # Tamanho do vocabulário (número de tokens únicos)
    n_pos: int = 1024     # Tamanho máximo da sequência para embeddings posicionais
    n_ctx: int = 1024     # Tamanho do contexto (comprimento máximo da sequência de entrada)
    n_embed: int = 768    # Dimensão dos embeddings e camadas ocultas
    n_layer: int = 12     # Número de camadas do transformer (blocos de atenção + feed-forward)
    n_head: int = 12      # Número de cabeças de atenção em cada camada
    n_inner: int = field(init=False)  # Dimensão da camada feed-forward, calculada no __post_init__

    # Hiperparâmetros de treinamento
    layer_norm_epsilon: float = 1e-5      # Epsilon para estabilidade numérica na normalização de camada
    initializer_range: float = 0.02       # Intervalo para inicialização aleatória dos pesos
    use_scaled_attention: bool = True     # Se True, usa atenção de produto escalar escalado

    # Configurações de treinamento
    batch_size: int = 32              # Número de amostras processadas em cada lote
    learning_rate: float = 1e-4       # Taxa de aprendizagem para o otimizador
    num_epochs: int = 10              # Número total de épocas de treinamento
    num_labels: int = 3               # Número de classes para classificação (se aplicável)

    # Regularização
    resid_pdrop: float = 0.1  # Taxa de dropout para saídas residuais
    embd_pdrop: float = 0.1   # Taxa de dropout para embeddings
    attn_pdrop: float = 0.1   # Taxa de dropout para atenção

    # Tokens especiais
    pad_token_id: int = 50256  # ID do token de padding
    bos_token_id: int = 50256  # ID do token de início de sequência (Beginning of Sequence)
    eos_token_id: int = 50256  # ID do token de fim de sequência (End of Sequence)

    # Otimização e checkpointing
    accumulation_steps: int = 4       # Número de passos para acumulação de gradiente
    patience: int = 3                 # Número de épocas para esperar antes de early stopping
    checkpoint_dir: str = "checkpoints"  # Diretório para salvar checkpoints do modelo
    log_dir: str = "logs"             # Diretório para salvar logs de treinamento

    # Identificação do modelo
    hub_model_id: str = None # ID do modelo no Hugging Face Hub

    def __post_init__(self):
        # Calcula a dimensão interna da camada feed-forward como 4 * n_embed
        # Esta é uma prática comum em arquiteturas GPT
        self.n_inner = 4 * self.n_embed

class GPT2(nn.Module):

    def __init__(self, config: 'GPTConfig'):
        super().__init__()
        self.config = config
        self.transformer = Transformer(config)
        self.readout_head = LinearReadoutHead(self.transformer.wte.weight, config)

    def set_tied(self):
        """
        Vincula os pesos da camada de embedding com a cabeça de leitura linear.
        Isso implementa o compartilhamento de pesos entre a entrada e a saída do modelo.
        """
        self.readout_head.set_embedding_weights(self.transformer.wte.weight)

    def forward(
        self,
        input_ids: torch.LongTensor,
        position_ids: torch.LongTensor = None,
        token_type_ids: torch.LongTensor = None,
        past: torch.Tensor = None
    ) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Realiza a passagem para frente do modelo GPT-2.

        :param input_ids: IDs dos tokens de entrada
        :param position_ids: IDs das posições (opcional)
        :param token_type_ids: IDs dos tipos de token (opcional)
        :param past: Estado passado para geração incremental (opcional)
        :return: Uma tupla contendo os logits de saída e os estados presentes
        """
        # Passa a entrada pelo transformer
        hidden_states, presents = self.transformer(input_ids, position_ids, token_type_ids, past)

        # Passa os estados ocultos pela cabeça de leitura linear
        logits = self.readout_head(hidden_states)

        return logits, presents

from typing import Optional

class GPT2ForSequenceClassification(nn.Module):
    def __init__(self, config: 'GPTConfig'):
        super().__init__()
        self.config = config
        self.num_labels = config.num_labels
        self.transformer = Transformer(config)
        self.dropout = nn.Dropout(config.resid_pdrop)
        self.classifier = nn.Linear(config.n_embed, self.num_labels)
        self._init_weights(self.classifier)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
    ):
        transformer_outputs = self.transformer(
            input_ids,
            token_type_ids=token_type_ids,
            position_ids=position_ids
        )

        hidden_states = transformer_outputs[0]  # [batch_size, sequence_length, hidden_size]

        # Determine as dimensões do batch e da sequência
        if input_ids is not None:
            batch_size, sequence_length = input_ids.shape[:2]
        else:
            batch_size, sequence_length = inputs_embeds.shape[:2]

        # Determine os comprimentos reais das sequências
        if attention_mask is not None:
            # Use a máscara de atenção para determinar o último token não-mascarado
            sequence_lengths = attention_mask.sum(dim=1) - 1
        elif self.config.pad_token_id is not None and input_ids is not None:
            # Se não temos máscara de atenção, mas temos um token de padding, use-o
            sequence_lengths = torch.ne(input_ids, self.config.pad_token_id).sum(-1) - 1
        else:
            # Se não temos nem máscara de atenção nem token de padding, assume que todas as sequências têm o mesmo comprimento
            sequence_lengths = torch.tensor([sequence_length - 1] * batch_size, device=hidden_states.device)

        # Garanta que sequence_lengths está no intervalo correto
        sequence_lengths = torch.clamp(sequence_lengths, min=0, max=sequence_length - 1)

        # Selecione o último estado oculto não-padding para cada sequência
        pooled_output = hidden_states[torch.arange(batch_size, device=hidden_states.device), sequence_lengths]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                # Regressão
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.squeeze(), labels.squeeze())
            else:
                # Classificação
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        output = (logits,) + transformer_outputs[1:]
        return ((loss,) + output) if loss is not None else output

    def freeze_transformer(self, n_blocks=None):
        for param in self.transformer.parameters():
            param.requires_grad = False

        if n_blocks:
            for i in range(1, n_blocks + 1):
                for param in self.transformer.h[-i].parameters():
                    param.requires_grad = True

    def unfreeze_transformer(self):
        for param in self.transformer.parameters():
            param.requires_grad = True

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup
from huggingface_hub import HfApi, HfFolder
from huggingface_hub import HFSummaryWriter
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import os
from huggingface_hub import HfApi
from huggingface_hub import HfFolder
from huggingface_hub import HFSummaryWriter

def train_gpt(model, train_loader, val_loader, config, device):
    print(f"Setting device to {device}")
    model.to(device)

    num_training_steps = len(train_loader) * config.num_epochs
    num_warmup_steps = int(0.1 * num_training_steps)

    optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )

    best_val_loss = float('inf')
    accumulation_steps = config.accumulation_steps
    patience_counter = 0

    # Set up Hugging Face Hub API
    api = HfApi()
    token = HfFolder.get_token()
    if token is None:
        raise ValueError("No Hugging Face token found. Please login using `huggingface-cli login`")

    # Set up HFSummaryWriter
    writer = HFSummaryWriter(logdir=f"runs/{config.checkpoint_dir}", repo_id=config.hub_model_id, commit_every=5)

    global_step = 0
    for epoch in range(config.num_epochs):
        model.train()
        total_loss = 0
        all_preds = []
        all_labels = []

        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{config.num_epochs}")

        for step, batch in progress_bar:
            input_ids, label_ids = [b.to(device) for b in batch]

            optimizer.zero_grad()

            outputs = model(input_ids, labels=label_ids)
            loss, logits = outputs[:2]
            loss = loss / accumulation_steps
            loss.backward()

            if (step + 1) % accumulation_steps == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

            total_loss += loss.item() * accumulation_steps

            # Collect predictions and labels for metrics
            preds = torch.argmax(logits, dim=-1).cpu().numpy()
            labels = label_ids.cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)

            global_step = epoch * len(train_loader) + step

            if global_step % 50 == 0:
                # Log metrics every batch
                writer.add_scalar("Loss/train", loss.item(), global_step)
                writer.add_scalar("Learning_rate", scheduler.get_last_lr()[0], global_step)

                # Calculate and log batch-level metrics
                batch_accuracy = accuracy_score(labels, preds)
                batch_f1 = f1_score(labels, preds, average='weighted')
                writer.add_scalar("Accuracy/train", batch_accuracy, global_step)
                writer.add_scalar("F1_Score/train", batch_f1, global_step)

            # Update progress bar
            progress_bar.set_postfix({
                'loss': f"{total_loss/(step+1):.4f}",
                'acc': f"{batch_accuracy:.4f}",
                'f1': f"{batch_f1:.4f}"
            })

            # Intermediate checkpoint and upload to Hugging Face Hub
            if (step + 1) % 1000 == 0:
                checkpoint_path = f"{config.checkpoint_dir}/checkpoint_step{global_step}.pt"
                torch.save({
                    'epoch': epoch,
                    'global_step': global_step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': total_loss,
                }, checkpoint_path)

                # Upload checkpoint to Hugging Face Hub
                api.upload_file(
                    path_or_fileobj=checkpoint_path,
                    path_in_repo=f"{config.checkpoint_dir}/checkpoint_step{global_step}.pt",
                    repo_id=config.hub_model_id,
                    token=token
                )
                print(f"Uploaded checkpoint to {config.hub_model_id}")

                # Evaluation
                val_loss, val_accuracy, val_f1 = evaluate(model, val_loader, device)

                # Log validation metrics
                writer.add_scalar("Loss/validation", val_loss, global_step)
                writer.add_scalar("Accuracy/validation", val_accuracy, global_step)
                writer.add_scalar("F1_Score/validation", val_f1, global_step)

                print(f"Val Loss: {val_loss:.4f}, "
                      f"Val Acc: {val_accuracy:.4f}, "
                      f"Val F1: {val_f1:.4f}")

        # Calculate epoch-level metrics
        avg_train_loss = total_loss / len(train_loader)
        train_accuracy = accuracy_score(all_labels, all_preds)
        train_f1 = f1_score(all_labels, all_preds, average='weighted')

        print(f"Epoch {epoch+1}/{config.num_epochs}, "
              f"Train Loss: {avg_train_loss:.4f}, "
              f"Train Acc: {train_accuracy:.4f}, "
              f"Train F1: {train_f1:.4f}")

        # Evaluation
        val_loss, val_accuracy, val_f1 = evaluate(model, val_loader, device)

        # Log validation metrics
        writer.add_scalar("Loss/validation", val_loss, global_step)
        writer.add_scalar("Accuracy/validation", val_accuracy, global_step)
        writer.add_scalar("F1_Score/validation", val_f1, global_step)

        print(f"Val Loss: {val_loss:.4f}, "
              f"Val Acc: {val_accuracy:.4f}, "
              f"Val F1: {val_f1:.4f}")

        # Save the best model and upload to Hugging Face Hub
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_path = f"{config.checkpoint_dir}/best_model.pt"
            torch.save(model.state_dict(), best_model_path)

            api.upload_file(
                path_or_fileobj=best_model_path,
                path_in_repo=f"{config.checkpoint_dir}_best_model.pt",
                repo_id=config.hub_model_id,
                token=token
            )
            print(f"Uploaded best model to {config.hub_model_id}")
        else:
            patience_counter += 1
            if patience_counter >= config.patience:
                print("Early stopping triggered")
                break

    print("Training completed!")
    writer.close()

def evaluate(model, val_loader, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids, label_ids = [b.to(device) for b in batch]

            outputs = model(input_ids, labels=label_ids)
            loss, logits = outputs[:2]

            total_loss += loss.item()

            preds = torch.argmax(logits, dim=-1).cpu().numpy()
            labels = label_ids.cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)

    avg_loss = total_loss / len(val_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    return avg_loss, accuracy, f1

def load_weight(model, state_dict):
    old_keys = []
    new_keys = []
    for key in state_dict.keys():
        new_key = None
        if key.endswith(".g"):
            new_key = key[:-2] + ".weight"
        elif key.endswith(".b"):
            new_key = key[:-2] + ".bias"
        elif key.endswith(".w"):
            new_key = key[:-2] + ".weight"
        if new_key:
            old_keys.append(key)
            new_keys.append(new_key)
    for old_key, new_key in zip(old_keys, new_keys):
        state_dict[new_key] = state_dict.pop(old_key)

    # Mapeamento de nomes de camadas
    layer_map = {
        "wte": "transformer.wte",
        "wpe": "transformer.wpe",
        "h": "transformer.h",
        "ln_f": "transformer.ln_f",
    }

    # Ajusta os nomes das chaves no state_dict
    for old_key in list(state_dict.keys()):
        parts = old_key.split('.')
        if parts[0] in layer_map:
            new_key = layer_map[parts[0]] + '.' + '.'.join(parts[1:])
            state_dict[new_key] = state_dict.pop(old_key)

    # Carrega os pesos no modelo
    model.load_state_dict(state_dict, strict=False)

    return model

from torch.utils.data import Dataset
from datasets import load_dataset

class SetenceClassificationDataset(Dataset):
    def __init__(self, split, tokenizer, max_length=None, pad_token_id=50256):
        self.dynasent_r1 = load_dataset("dynabench/dynasent", 'dynabench.dynasent.r1.all', trust_remote_code=True)
        self.dynasent_r2 = load_dataset("dynabench/dynasent", 'dynabench.dynasent.r2.all')
        self.sst = load_dataset("SetFit/sst5")
        self.tokenizer = tokenizer

        def convert_sst_label(s):
            return s.split(" ")[-1]

        self.dataset = {}

        self.label_ids = {
            "neutral": 0,
            "positive": 1,
            "negative": 2
        }

        self.dataset["features"] = []
        self.dataset["label_ids"] = []

        self.dataset["features"].extend([tokenizer.encode(s) for s in self.dynasent_r1[split]["sentence"]])
        self.dataset["features"].extend([tokenizer.encode(s) for s in self.dynasent_r2[split]["sentence"]])
        self.dataset["features"].extend([tokenizer.encode(s) for s in self.sst[split]["text"]])

        self.dataset["label_ids"].extend([self.label_ids[l] for l in self.dynasent_r1[split]["gold_label"]])
        self.dataset["label_ids"].extend([self.label_ids[l] for l in self.dynasent_r2[split]["gold_label"]])
        self.dataset["label_ids"].extend([self.label_ids[convert_sst_label(l)] for l in self.sst[split]["label_text"]])

        self.max_length = max_length or self._longest_encoded_length()

        self.dataset["padded_features"] = [
            features[:self.max_length] +
            [pad_token_id] * (self.max_length - len(features))
            for features in self.dataset["features"]
        ]

    def __getitem__(self, index):
        encoded = self.dataset["padded_features"][index]
        label = self.dataset["label_ids"][index]

        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

    def __len__(self):
        return len(self.dataset["padded_features"])

    def _longest_encoded_length(self):
        return max(len(features) for features in self.dataset["features"])

import tiktoken

tokenizer = tiktoken.encoding_for_model("gpt-2")

train_dataset = SetenceClassificationDataset("train", tokenizer)
val_dataset = SetenceClassificationDataset("validation", tokenizer)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=48,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=48,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

import requests
from huggingface_hub import notebook_login

notebook_login()

def downloadGPT2Checkpoint():
    if not os.path.exists('./gpt2-pytorch_model.bin'):
        print("Downloading GPT-2 checkpoint...")
        url = 'https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin'
        r = requests.get(url, allow_redirects=True)
        open('gpt2-pytorch_model.bin', 'wb').write(r.content)
    assert os.path.exists("./gpt2-pytorch_model.bin")

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Exemplo de uso
config = GPTConfig(
    hub_model_id="diegomrodrigues/GPT2-Fine-Tuned-Classification",
    batch_size=48,
    learning_rate=3e-5,
    num_epochs=50,
    accumulation_steps=2,
    patience=10,
    checkpoint_dir="checkpoints_5blocks_from_32904_7380"
)

model = GPT2ForSequenceClassification(config)

#state_dict = torch.load('/content/checkpoints_3blocks_from_32904_7380/checkpoint_step999.pt')
#model.load_state_dict(state_dict['model_state_dict'])

#model.freeze_transformer(n_blocks=5)

#train_gpt(model, train_loader, val_loader, config, device)


Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
